# Improvements to models

We attempt to modify the model in the following ways:  

1.  Generalizing to a second order model  (first order for leaders/predators and second order for followers/prey)  

2. Adding perception cones of individuals 

3. Change the area of effect of other individuals: the interaction force is no longer averaged over the entire population but over $k$ nearest neighbours

## 1 A second order model  

The following is a second order model  

$$
\begin{equation}
    \begin{cases}
        \dot{x}_j = v_j\\[2ex]
        \dot{v}_j = \dfrac1N \sum\limits_{k=1}^N  \bigg\{\dfrac{\kappa_h(v_k-v_j)}{(\zeta_h ^2 + \|x_j-x_k\|^2)^{\gamma_h}} - a(x_j-x_k) + b_1\dfrac{x_j-x_k}{\|x_j-x_k\|^2}\bigg\} + {\color{red}b_2\dfrac{x_j-z}{\|x_j-z\|^2}} \\[5ex]
        \dot{z} = w\\[2ex]
        \dot{w} = \dfrac{1}{N} \sum\limits_{k=1}^N  \dfrac{\kappa_p(v_k-w)}{(\zeta_p ^2 + \|z-x_k\|^2)^{\gamma_p}} + c\dfrac{x_k-z}{\|x_k-z\|^p}
    \end{cases}
\end{equation}
$$  

For the moment $\zeta=\gamma=1$ for both the prey and predator, while the parameters $a,b_1, b_2, c, p$ can be tuned.  
It might be better if the summation above was taken to exclude the $j^{th}$ prey.  

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from solvers.rk_solvers import *
from solvers.euler import *
from helper import *

In [2]:
def model1(y, params):
  '''
  Returns ndarray of shape (dim, 2) containing the derivatives.   
  
  Input:   
    - y: an array containing the following inputs   
      - pprey: ndarray of (N,2), the positions of the prey  
      - vprey: ndarray of (N,2), the velocities of the prey  
      - ppred: ndarray of (1,2), the positions of the predator  

    - params: py dict, extra parameters for the model (N, kh, a, b1, b2, c, p) for now

  Returns:   
    - dy: ndarray of (2*N+1, 2), the derivatives of the inputs  
  '''  

  # unpacking parameters
  N, kh, a, b1, b2, c, p = params.values() 

  pprey = y[0:N]  
  vprey = y[N:2*N]  
  ppred = y[2*N:]
  # vpred = y[2*N+1:]  

  # List of len N, ndarrays of shape (N-1, 2)
  dist_prey =  np.array([pprey[i] - np.delete(pprey, i, axis = 0) for i in range(N)])  
  
  vel_prey  =  np.array([vprey[i] - np.delete(vprey, i, axis = 0) for i in range(N)])

  # kernel of prey
  ker_prey = kh * np.sum (-1 * vel_prey / (1 + (np.linalg.norm(dist_prey, axis = 2)[:,:,np.newaxis]**2))
                    ,axis = 1)   
  
  # attraction of prey
  attrac_prey = -1 * a * np.sum(dist_prey, axis = 1)  

  # repulsion of prey
  rep_prey = b1 * np.sum (dist_prey / (np.linalg.norm(dist_prey, axis = 2)[:,:,np.newaxis]**2),
                          axis = 1)
                          
  # repulsion of predator
  rep_pred = b2 * (pprey - ppred) / (np.linalg.norm(pprey - ppred, axis = 1)[:,np.newaxis]**2)  

  # acceleration of prey
  acc_prey = (1/N) * (ker_prey + attrac_prey + rep_prey) + rep_pred  

  # velocity of predator
  vpred = (1/N) * (c * np.sum((pprey - ppred) / (np.linalg.norm(pprey - ppred, axis = 1)[:,np.newaxis]**p), 
                                 axis = 0))[np.newaxis]
                                 
  return np.concatenate((vprey, acc_prey, vpred))

### 1.1 Testing

For testing, we use values of $\kappa_h = 0.5, a=1, b_1 = 1, b_2 = 0.07, c = 1.5 ,p = 1.2$.  

An observation is that larger $p$ will lead to predators moving away (inversely proportional to distance)

In [42]:
N = 200

case1  = computeSoln(model1, 
    {'no. of prey': N, 
    'kappa for prey': 0.5, 
    'attraction of prey': 1, 
    'repulsion of prey': 1, 
    'repulsion of pred': 0.07, 
    'attraction of pred': 1.5, 
    'exponent of dist pred': 1.2},
    1000,
    (0,100),
    False,
    'random',
    'rk2',
    [0.0, 0.6, 1.2, 1.8, 2.4, 3.0, 3.6, 4.0],
    12)

No. of time steps: 1000


100%|██████████| 999/999 [00:18<00:00, 54.45it/s]


In [43]:
case1[2] = [i*10  for i in range(10)]

In [44]:
multiPlot(case1, None, True)

## 2 Adding the perception cones  

This is the same model but each individual of the prey can only react to the changes within their perceptioin cones.  

$$
\begin{equation}
    \begin{cases}
        \dot{x}_j = v_j\\[2ex]
        \dot{v}_j = \dfrac1N \sum\limits_{k=1}^N  \psi_\alpha(x_j,x_k,v_j) \bigg\{\dfrac{\kappa_h(v_k-v_j)}{(\zeta_h ^2 + \|x_j-x_k\|^2)^{\gamma_h}} - a(x_j-x_k) + b_1\dfrac{x_j-x_k}{\|x_j-x_k\|^2}\bigg\} + {\color{red}b_2\dfrac{x_j-z}{\|x_j-z\|^2}} \\[5ex]
        \dot{z} = w\\[2ex]
        \dot{w} = \dfrac{1}{N} \sum\limits_{k=1}^N  \dfrac{\kappa_p(v_k-w)}{(\zeta_p ^2 + \|z-x_k\|^2)^{\gamma_p}} + c\dfrac{x_k-z}{\|x_k-z\|^p}
    \end{cases}
\end{equation}
$$  

## 3 Adding topological effect  

Finally we consider the nearest neighbours' effect on each prey.